# Demo of DEMs products<a name="top"></a>

*****

__This script is present products available through the [config_tool.ipynb](config_tool.ipynb) notebook__

This notebook present and demonstrate how to interacte with two DEMs products covering Switzerland:

**swissALTI3D** product named 'swissalti3d_scene' is restricted to Switzerland boundaries and is available at 2 m resolution (see https://www.swisstopo.admin.ch/en/geodata/height/alti3d.html for further information).

**ASTER GDEM v3** product named 'astgtmv003_scene' cover Switzerland geographical extent plus part of neighbouring countries at 30 m resolution. You will be able to visualize it in the next cells.

The script is structured as follows:
- **[ASTER GDEM v3](#ds_aster)**: Presenting how to interact with Aster GDEM v3.
- **[swissALTI3D](#ds_swiss)**: Presenting how to interact with swissALTI3D.
- **[reprojecting and resampling](#rere)**: Presenting how to reproject and resample high resolution swissALTI3D to medium resolution ASTER GDEM v3.

In [ ]:
# Make sure the script is using the proper kernel
try:
    %run ../swiss_utils/assert_env.py
except:
    %run ./swiss_utils/assert_env.py

In [ ]:
# Import modules

# reload module before executing code
%load_ext autoreload
%autoreload 2

# define modules locations (you might have to adapt define_mod_locs.py)
%run ../swiss_utils/define_mod_locs.py

import os
import glob

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import ipywidgets as widgets
import richdem as rd

from datetime import datetime, timedelta
from shapely.geometry import Polygon
from math import radians, cos, sin, asin, sqrt
from IPython.display import display, HTML
from scipy.stats import linregress

from swiss_utils.data_cube_utilities.sdc_utilities import new_get_query_metadata
from swiss_utils.data_cube_utilities.sdc_advutils import draw_map, oneband_fig, natural_sort, make_contact_sheet

import datacube
dc = datacube.Datacube()

In [ ]:
# Functions

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [ ]:
# First select a mountainous  AOI on Swiss border with France and Italy
# (to see the differences between the two DEM products and have nice visuals)

min_lon = 6.970808
max_lon = 7.119154
min_lat = 45.881054
max_lat = 45.967029

m, poly = draw_map((min_lat, max_lat), (min_lon, max_lon))
m

In [ ]:
# If you want, you can draw a different AOI by using the buttons on the above map and then run this cell

try:
    coords = poly.last_draw['geometry']['coordinates']
    geo_extent = Polygon(coords[0]).bounds

    min_lon = geo_extent[0]
    min_lat = geo_extent[1]
    max_lon = geo_extent[2]
    max_lat = geo_extent[3]
except:
    pass

## ASTER GDEM V3<a name="ds_aster"></a>
[<div style="text-align: right; font-size: 24px"> &#x1F51D; </div>](#top)

In [ ]:
# Identify product

product = 'astgtmv003_scene'

In [ ]:
# Visualize measurements available and use them all

measurement_list = dc.list_measurements(with_pandas=False)
measurements_for_product = filter(lambda x: x['product'] == product, measurement_list)
valid_measurements_name_array = set(map(lambda x: x['name'], measurements_for_product))
measurements = list(valid_measurements_name_array)

print(measurements)
print("""
elevation: elevation in meter\n\
slope: slope in degree\n\
aspect: orientation in degree (North == 0°)""")

In [ ]:
# Note on product CRS and resolution

products = dc.list_products()
crs = products[products['name'] == product].default_crs.values[0].crs_str
res_aster = products[products['name'] == product].default_resolution.values[0][1]

mtd = new_get_query_metadata(dc, product)
res_aster_m = haversine(np.mean(mtd['lat_extents']),
                  np.mean(mtd['lon_extents']) - (res_aster / 2),
                  np.mean(mtd['lat_extents']),
                  np.mean(mtd['lon_extents']) + (res_aster / 2)) * 1000

display(HTML(f"""As you can see <a href="https://epsg.io/{crs.split(':')[1]}" target="_blank" >here</a> \
             the CRS units are in degree meaning the resolution of this product is {res_aster} degree, \
             corresponding to around {int(res_aster_m)} meter."""))

In [ ]:
# Note on time
# As all products based on a "one time" dataset, the 'time = [start_date, end_date]' parameter is not
# required in dc.load function. But you can use this parameter as long as it includes the product date range.

min_max_dates = mtd['time_extents']

print(min_max_dates)

In [ ]:
# Load the product into an xarray.Dataset

ds_aster = dc.load(product = product,
                   time = [min_max_dates[0],                      # start_date is inclusive (<=)
                           min_max_dates[1] + timedelta(days=1)], # end_date is not inclusive (<)
                   lon = (min_lon, max_lon), lat = (min_lat, max_lat),
                   measurements = measurements)
# remove nodata
ds_aster = ds_aster.where(ds_aster != ds_aster[list(ds_aster.data_vars)[0]].nodata)
if len(set(ds_aster.dims).intersection(['x', 'y'])) >= 1:
    ds_aster = ds_aster.rename({'x': 'longitude', 'y': 'latitude'})

print(ds_aster)

In [ ]:
# Display dataset

tmp_prfx = 'tmp_fig_'

for filename in glob.glob(f"{tmp_prfx}*.png"):
    os.remove(filename) 

for m in ds_aster.data_vars:
    oneband_fig(ds_aster.isel(time=0)[m],
            leg = 'gist_rainbow',
            title = m,
            fig_name = f"{tmp_prfx}{m}.png",
            max_size = 7)
    
make_contact_sheet(natural_sort(glob.glob(f"{tmp_prfx}*.png")), ncols = 2, photow = 700,
                   title = product, size = 24)

## swissALTI3D<a name="ds_swiss"></a>
[<div style="text-align: right; font-size: 24px"> &#x1F51D; </div>](#top)

In [ ]:
# Identify product

product = 'swissalti3d_scene'

In [ ]:
# Visualize measurements available and use them all

measurement_list = dc.list_measurements(with_pandas=False)
measurements_for_product = filter(lambda x: x['product'] == product, measurement_list)
valid_measurements_name_array = set(map(lambda x: x['name'], measurements_for_product))
measurements = list(valid_measurements_name_array)

print(measurements)
print("""
elevation: elevation in meter\n\
slope: slope in degree\n\
aspect: orientation in degree (North == 0°)""")

In [ ]:
# Note on product CRS and resolution

products = dc.list_products()
crs = str(products[products['name'] == product].default_crs.values[0])
res_swiss = products[products['name'] == product].default_resolution.values[0][1]

display(HTML(f"""As you can see <a href="https://epsg.io/{crs.split(':')[1]}" target="_blank" >here</a> \
             the CRS units are in metre meaning the resolution of this product is {res_swiss} metre. \
             Meaning it will quickly use a lot of memory and time to load. \
             <br>This is the reason why we are using a small AOI."""))

In [ ]:
# Load the product into an xarray.Dataset, this time without using the time parameter

ds_swiss = dc.load(product = product,
                   lon = (min_lon, max_lon), lat = (min_lat, max_lat),
                   measurements = measurements)
# remove nodata
ds_swiss = ds_swiss.where(ds_swiss != ds_swiss[list(ds_swiss.data_vars)[0]].nodata)
if len(set(ds_swiss.dims).intersection(['x', 'y'])) >= 1:
    ds_swiss = ds_swiss.rename({'x': 'longitude', 'y': 'latitude'})

print(ds_swiss)

# This time remove the single time dimension
ds_swiss = ds_swiss.isel(time=0)

In [ ]:
# Display dataset

tmp_prfx = 'tmp_fig_'

for filename in glob.glob(f"{tmp_prfx}*.png"):
    os.remove(filename) 

for m in ds_swiss.data_vars:
    oneband_fig(ds_swiss[m],
            leg = 'gist_rainbow',
            title = m,
            fig_name = f"{tmp_prfx}{m}.png",
            max_size = 7)
    
make_contact_sheet(natural_sort(glob.glob(f"{tmp_prfx}*.png")), ncols = 2, photow = 700,
                   title = product, size = 24)

## Reprojecting and resampling<a name="rere"></a>

Let's imagine we need to compare or combine the swissALTI3d product with the ASTER GDEM product by reprojecting and resampling the first one.

As presented in [demo_FUN_dcload.pynb](demo_FUN_dcload.ipynb) there are several way to change the initial projection and resolution of a given product, but in any GIR/RS context reprojection need to be handled carefully. Here follow some possible attemps from worse to best.

### 1. Directly load using the like option
[<div style="text-align: right; font-size: 24px"> &#x1F51D; </div>](#top)

In [ ]:
# Load the product into an xarray.Dataset, this time without using the time parameter

ds_like = dc.load(product = product,
#                   resampling = 'med',
                  like = ds_aster)
# remove nodata
ds_like = ds_like.where(ds_like != ds_like[list(ds_like.data_vars)[0]].nodata)
if len(set(ds_like.dims).intersection(['x', 'y'])) >= 1:
    ds_like = ds_like.rename({'x': 'longitude', 'y': 'latitude'})

print(ds_like)

# remove the single time dimension
ds_like = ds_like.isel(time=0)

In [ ]:
# Display dataset

tmp_prfx = 'tmp_fig_'

for filename in glob.glob(f"{tmp_prfx}*.png"):
    os.remove(filename) 

for m in ds_like.data_vars:
    oneband_fig(ds_like[m],
            leg = 'gist_rainbow',
            title = m,
            fig_name = f"{tmp_prfx}{m}.png",
            max_size = 7)
    
make_contact_sheet(natural_sort(glob.glob(f"{tmp_prfx}*.png")), ncols = 2, photow = 700,
                   title = product, size = 24)

If you compare the contact sheet you just created with the ASTER GDEM v3 contact sheet above (you can add a `fig_name` parameter in each `make_contact_sheet` and re-run the two related cells to save a figure for easier comparison) you will see the `ds_like`one is less smooth. You can look at the scatter plot of each variable of both dataset using the two cells below.

In [ ]:
# Select one variable and run the next cell to visualize how they respectivelly correlate

var_sel = widgets.RadioButtons(options=list(ds_aster.data_vars), disabled=False)
display(widgets.Label('Select a product and run the next cell: '), var_sel)

In [ ]:
varx = ds_aster[var_sel.value].values.flatten()
vary = ds_like[var_sel.value].values.flatten()
mask = ~np.isnan(varx) & ~np.isnan(vary)
slope, intercept, r_value, p_value, std_err = linregress(varx[mask], vary[mask])
print(f"{var_sel.value}:\nslope = {slope}\nintercept = {intercept}\nr_value = {r_value}")

plt.scatter(varx, vary)
plt.show()

You probably notice elevation is quite well correlated (r_value should be 0.999 if you did not modified the demo AOI), but it gets worse with slope (0.668) and really bad with aspect (0.593).
This is due to the fact the default resampling method used by dc.load is 'nearest'. Meaning the value of the 30 m resampled pixel will correspond to the value of the original 2 m pixel at its center !

Create and run a cell with `dc.load?` to access the documentation of `dc.load`function, and have a look at it. Then move back at [the beginning of this chapter](#rere), uncomment the `resampling = 'med',` parameter in dc.load function, then go through the cells. This time you should get a smoother result with slighly better correlations (r_value for elevation still at 0.999, slope at 0.734 and aspect at 0.690) as resampled 30 m values will correspond to the median value of all overlaping 2 m values.

But correlations of slope and aspect are still very weak. Then we need more complex processing.

### 2. Computing slope and aspect from elevation
As the units of elevation value is meter, we need to stay in meter for latitude and longitude to calculate slope. Then the most logical way would be to resample and compute in the original CRS (in meter) and then reproject in the `ds_aster` CRS (in decimal degree). Which would be fine if we would not need to perfectly overlay the two DEM dataset (then requiring the exact same resolution and AOI) for comparison.

Then an inverse approach will be more easy to implement by first reprojecting and resampling the original elevation, and then compute slope and aspect using richdem package (https://richdem.readthedocs.io/en/latest/python_api.html#richdem.TerrainAttribute) appying a conversion factor `zscale` based on the center of the AOI (as we prviously did to convert the ASTED GDEM v3 resolution from dd to m at the beginning of this notebook). Keeping in mind such approach is only valid in a small AOI.
[<div style="text-align: right; font-size: 24px"> &#x1F51D; </div>](#top)

In [ ]:
# Load the product into an xarray.Dataset, this time loading elevation only with a median resampling

ds_like = dc.load(product = product,
                  measurements = ['elevation'],
                  resampling = 'med',
                  like = ds_aster)
# remove nodata
ds_like = ds_like.where(ds_like != ds_like[list(ds_like.data_vars)[0]].nodata)
if len(set(ds_like.dims).intersection(['x', 'y'])) >= 1:
    ds_like = ds_like.rename({'x': 'longitude', 'y': 'latitude'})

print(ds_like)

# remove the single time dimension
ds_like = ds_like.isel(time=0)

In [ ]:
# Then compute slope and aspect

rd_tmp = xr.DataArray(rd.TerrainAttribute(rd.rdarray(ds_like.elevation, no_data = -9999),
                                          attrib='slope_degrees',
                                          zscale = res_aster * res_aster_m),
                      coords={'latitude': ds_like.latitude.values,
                              'longitude': ds_like.longitude.values},
                      dims=('latitude', 'longitude'))
ds_like = ds_like.merge(rd_tmp.to_dataset(name = 'slope'))

rd_tmp = xr.DataArray(rd.TerrainAttribute(rd.rdarray(ds_like.elevation, no_data = -9999),
                                          attrib='aspect',
                                          zscale = res_aster * res_aster_m),
                      coords={'latitude': ds_like.latitude.values,
                              'longitude': ds_like.longitude.values},
                      dims=('latitude', 'longitude'))
ds_like = ds_like.merge(rd_tmp.to_dataset(name = 'aspect'))


In [ ]:
# Display dataset

tmp_prfx = 'tmp_fig_'

for filename in glob.glob(f"{tmp_prfx}*.png"):
    os.remove(filename) 

for i, m in enumerate(ds_like.data_vars):
    oneband_fig(ds_like[m],
            leg = 'gist_rainbow',
            title = m,
            fig_name = f"{tmp_prfx}{i}.png",
            max_size = 7)
    
make_contact_sheet(natural_sort(glob.glob(f"{tmp_prfx}*.png")), ncols = 2, photow = 700,
                   title = product)

In [ ]:
# Select one variable and run the next cell to visualize how they respectivelly correlate

var_sel = widgets.RadioButtons(options=list(ds_aster.data_vars), disabled=False)
display(widgets.Label('Select a product and run the next cell: '), var_sel)

In [ ]:
varx = ds_aster[var_sel.value].values.flatten()
vary = ds_like[var_sel.value].values.flatten()
mask = ~np.isnan(varx) & ~np.isnan(vary)
slope, intercept, r_value, p_value, std_err = linregress(varx[mask], vary[mask])
print(f"{var_sel.value}:\nslope = {slope}\nintercept = {intercept}\nr_value = {r_value}")

plt.scatter(varx, vary)
plt.show()


As you can see correlations are a bit better still approximative for slope and aspect, which is normal as we are smoothing the elevation layer by applying the median method.

Moreover you can see we created string artefacts in the computed slope layer.